# FlowSort II (Promethee II)
This module shows an example of how to use the "FlowSort II" module.

## Definition of inputs and problem formalization

In [1]:
import pandas as pd
from modular_parts.preference import compute_preference_indices
from modular_parts.flows import calculate_promethee_outranking_flows, calculate_net_outranking_flows
from modular_parts.sorting import calculate_flowsortII_sorted_alternatives
from core.enums import Direction, GeneralCriterion, FlowType, CompareProfiles


alternatives = [f"a{i}" for i in range(1, 8)]
profiles = [f"p{i}" for i in range(1, 4)]
criteria = [f"c{i}" for i in range(1, 4)]
criteria_directions = pd.Series([Direction.MAX, Direction.MIN, Direction.MAX], index=criteria)
criteria_weights = pd.Series([0.3, 0.2, 0.5], index=criteria)
generalised_criteria = pd.Series([GeneralCriterion.U_SHAPE,
                                  GeneralCriterion.V_SHAPE_INDIFFERENCE,
                                  GeneralCriterion.USUAL], index=criteria)
preference_thresholds = pd.Series([2, 10, None], index=criteria)
indifference_thresholds = pd.Series([1, 5, None], index=criteria)
standard_deviations = pd.Series([None, None, None], index=criteria) # None, because we do not use GeneralCriterion.GAUSSIAN

alternatives_performances = pd.DataFrame([[15, 83, 21],
                                          [10, 90, 15],
                                          [11, 75, 20],
                                          [18, 59, 20],
                                          [17, 60, 28],
                                          [22, 44, 15],
                                          [13, 62, 22]], index=alternatives, columns=criteria)
profiles_performances = pd.DataFrame([[12, 80, 16],
                                      [15, 68, 18],
                                      [18, 55, 20]], index=profiles, columns=criteria)

central_categories = [f"C{i}" for i in range(1, 4)]
boundary_categories = [f"C{i}" for i in range(1, 5)]
limiting_categories = [f"C{i}" for i in range(1, 3)]

## Required Data

### Required Preferences

In [2]:
aggregated_altenatives_vs_profiles_preferences, _ = compute_preference_indices(alternatives_performances,
                                                                    preference_thresholds,
                                                                    indifference_thresholds,
                                                                    standard_deviations,
                                                                    generalised_criteria,
                                                                    criteria_directions,
                                                                    criteria_weights,
                                                                    profiles_performances)

aggregated_profiles_vs_profiles_preferences, _ =compute_preference_indices(profiles_performances,
                                                                    preference_thresholds,
                                                                    indifference_thresholds,
                                                                    standard_deviations,
                                                                    generalised_criteria,
                                                                    criteria_directions,
                                                                    criteria_weights)

### Required Flows

In [3]:
profile_based_flows = calculate_promethee_outranking_flows(aggregated_altenatives_vs_profiles_preferences,
                                                           flow_type=FlowType.PROFILE_BASED,
                                                           profiles_preferences=aggregated_profiles_vs_profiles_preferences)
profile_based_flows

positive  negative
Ra1 p1  0.000000  0.933333
    p2  0.400000  0.500000
    p3  0.833333  0.166667
    a1  0.600000  0.233333
Ra2 p1  0.333333  0.666667
    p2  0.666667  0.333333
    p3  1.000000  0.000000
    a2  0.000000  1.000000
Ra3 p1  0.000000  0.833333
    p2  0.460000  0.500000
    p3  0.833333  0.000000
    a3  0.333333  0.293333
Ra4 p1  0.000000  1.000000
    p2  0.333333  0.653333
    p3  0.666667  0.000000
    a4  0.653333  0.000000
Ra5 p1  0.000000  1.000000
    p2  0.333333  0.640000
    p3  0.666667  0.166667
    a5  0.806667  0.000000
Ra6 p1  0.166667  0.833333
    p2  0.500000  0.500000
    p3  0.833333  0.166667
    a6  0.500000  0.500000
Ra7 p1  0.000000  0.900000
    p2  0.433333  0.513333
    p3  0.793333  0.166667
    a7  0.580000  0.226667

In [4]:
profile_based_flows = calculate_net_outranking_flows(profile_based_flows, True)
profile_based_flows

positive  negative       net
Ra1 p1  0.000000  0.933333 -0.933333
    p2  0.400000  0.500000 -0.100000
    p3  0.833333  0.166667  0.666667
    a1  0.600000  0.233333  0.366667
Ra2 p1  0.333333  0.666667 -0.333333
    p2  0.666667  0.333333  0.333333
    p3  1.000000  0.000000  1.000000
    a2  0.000000  1.000000 -1.000000
Ra3 p1  0.000000  0.833333 -0.833333
    p2  0.460000  0.500000 -0.040000
    p3  0.833333  0.000000  0.833333
    a3  0.333333  0.293333  0.040000
Ra4 p1  0.000000  1.000000 -1.000000
    p2  0.333333  0.653333 -0.320000
    p3  0.666667  0.000000  0.666667
    a4  0.653333  0.000000  0.653333
Ra5 p1  0.000000  1.000000 -1.000000
    p2  0.333333  0.640000 -0.306667
    p3  0.666667  0.166667  0.500000
    a5  0.806667  0.000000  0.806667
Ra6 p1  0.166667  0.833333 -0.666667
    p2  0.500000  0.500000  0.000000
    p3  0.833333  0.166667  0.666667
    a6  0.500000  0.500000  0.000000
Ra7 p1  0.000000  0.900000 -0.900000
    p2  0.433333  0.513333 -0.080000
    p3  0.793333  0.166667  0.626667
    a7  0.580000  0.226667  0.353333

## Usage of FlowSort II

### Central Profiles

In [5]:
sorted_alternatives = calculate_flowsortII_sorted_alternatives(central_categories,
                                                              profiles_performances,
                                                              criteria_directions,
                                                              profile_based_flows,
                                                              CompareProfiles.CENTRAL_PROFILES)
sorted_alternatives

,positive,negative,net
a1,C2,C3,C3
a2,C1,C1,C1
a3,C2,C2,C2
a4,C3,C3,C3
a5,C3,C3,C3
a6,C2,C2,C2
a7,C2,C3,C3


### Boundary Profiles

In [6]:
sorted_alternatives = calculate_flowsortII_sorted_alternatives(boundary_categories,
                                                              profiles_performances,
                                                              criteria_directions,
                                                              profile_based_flows,
                                                              CompareProfiles.BOUNDARY_PROFILES)
sorted_alternatives

,positive,negative,net
a1,C3,C3,C3
a2,C1,C1,C1
a3,C2,C3,C3
a4,C3,C3,C3
a5,C4,C4,C4
a6,C2,C2,C2
a7,C3,C3,C3


### Limiting Profiles

In [7]:
sorted_alternatives = calculate_flowsortII_sorted_alternatives(limiting_categories,
                                                              profiles_performances,
                                                              criteria_directions,
                                                              profile_based_flows,
                                                              CompareProfiles.LIMITING_PROFILES)
sorted_alternatives

,positive,negative,net
a1,C2,C2,C2
a2,Under limit,Under limit,Under limit
a3,C1,C2,C2
a4,C2,C2,C2
a5,Over limit,Over limit,Over limit
a6,C1,C1,C1
a7,C2,C2,C2
